## Code generation

In [1]:
# import sys
# !{sys.executable} -m pip install --upgrade pip
# !{sys.executable} -m pip install sympy
# !{sys.executable} -m pip install scipy
# !{sys.executable} -m pip install matplotlib

In [2]:
# Remove matplotlib deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [3]:
# initialize sympy printing (for latex output)
from sympy import init_printing, Symbol
init_printing()

# import functions and classes for compartment models
import sys
sys.path.insert(0, "./compartor") #use local copy
from compartor import *

Here, we illustrate how to export the moment equations in LaTeX format or generate code for simulations.

We consider again the exemplary model of the paper and derive moments equations with the automated function.

In [4]:
x = Content('x')
y = Content('y')
z = Content('z')

from sympy import symbols, log, exp, sin, cos, tan
kM,h = symbols('kM h') # The Michaelis-Menten constant and hill exponent

# define intake distribution
pi_I0 = OutcomeDistribution.Poisson(Symbol("\pi_{Poiss0}(y; \lambda)"), y[0], Symbol("\lambda"))
pi_I1 = OutcomeDistribution.Poisson(Symbol("\pi_{Poiss1}(y; \lambda)"), y[1], Symbol("\lambda"))
pi_I = OutcomeDistribution.CombineIndependent(Symbol("\pi_{Poiss}(y; \lambda)"), pi_I0, pi_I1)

Birth0         = TransitionClass( [x]       -to> [x+(1,0)],       'k_b',            name='b_0')
Death0         = TransitionClass( [x]       -to> [x+(-1,0)],      'k_d', g=x[0],    name='d_0')
Birth1         = TransitionClass( [x]       -to> [x+(0,1)],       'k_b',            name='b_1')
Death1         = TransitionClass( [x]       -to> [x+(0,-1)],      'k_d', g=x[1],    name='d_1')

Intake        = TransitionClass( {}        -to> [y],         'k_I',         pi=pi_I, name='I')
Exit          = TransitionClass( [x]       -to> {},          'k_E',                  name='E')

transitions = [Intake, Exit, Birth0, Death0, Birth1, Death1]
display_transition_classes(transitions)

EmptySet ---> [y],  h_I = \pi_{Poiss}(y; \lambda)*k_I
          [x] ---> EmptySet,  h_E = k_E*n(x)         
       [x] ---> [(1, 0) + x],  h_b_0 = k_b*n(x)      
    [x] ---> [(-1, 0) + x],  h_d_0 = k_d*n(x)*x[0]   
       [x] ---> [(0, 1) + x],  h_b_1 = k_b*n(x)      
    [x] ---> [(0, -1) + x],  h_d_1 = k_d*n(x)*x[1]   

In [5]:
M = Moment
moments = [
    M(0,0),
    M(0,0)**2,
    M(1,0),
    M(1,0)**2,
    M(0,1),
    M(0,1)**2,
    M(1,1),
]
display(moments)
equations = automated_moment_equations(2, transitions, moments, clna=True)
display_moment_equations(equations)

⎡                    2                            2                            2                    ⎤
⎣Moment(0, 0), Moment (0, 0), Moment(1, 0), Moment (1, 0), Moment(0, 1), Moment (0, 1), Moment(1, 1)⎦

(0) fM: Moment(0, 0)	pM: 1	pDM: DeltaM(0, 0)
(1) fM: Moment(0, 0)	pM: 1	pDM: DeltaM(0, 0)
(2) fM: Moment(0, 0)	pM: 1	pDM: DeltaM(0, 0)
(3) fM: Moment(0, 0)	pM: 1	pDM: DeltaM(0, 0)
(4) fM: Moment(0, 0)	pM: 1	pDM: DeltaM(0, 0)
(5) fM: Moment(0, 0)	pM: 1	pDM: DeltaM(0, 0)
(0) fM: Moment(0, 0)**2	pM: 1	pDM: DeltaM(0, 0)**2
(0) fM: Moment(0, 0)**2	pM: Moment(0, 0)	pDM: DeltaM(0, 0)
(1) fM: Moment(0, 0)**2	pM: 1	pDM: DeltaM(0, 0)**2
(1) fM: Moment(0, 0)**2	pM: Moment(0, 0)	pDM: DeltaM(0, 0)
(2) fM: Moment(0, 0)**2	pM: 1	pDM: DeltaM(0, 0)**2
(2) fM: Moment(0, 0)**2	pM: Moment(0, 0)	pDM: DeltaM(0, 0)
(3) fM: Moment(0, 0)**2	pM: 1	pDM: DeltaM(0, 0)**2
(3) fM: Moment(0, 0)**2	pM: Moment(0, 0)	pDM: DeltaM(0, 0)
(4) fM: Moment(0, 0)**2	pM: 1	pDM: DeltaM(0, 0)**2
(4) fM: Moment(0, 0)**2	pM: Moment(0, 0)	pDM: DeltaM(0, 0)
(5) fM: Moment(0, 0)**2	pM: 1	pDM: DeltaM(0, 0)**2
(5) fM: Moment(0, 0)**2	pM: Moment(0, 0)	pDM: DeltaM(0, 0)
(0) fM: Moment(1, 0)	pM: 1	pDM: DeltaM(1, 0)
(1) fM: Moment(1, 0)	pM: 

Computed moment equations for desired moments $\left< N\right> $, $\left< N^{2}\right> $, $\left< M^{\left(1, 0\right)}\right> $, $\left< {\left(M^{\left(1, 0\right)}\right)}^{\mathtt{\text{2}}}\right> $, $\left< M^{\left(0, 1\right)}\right> $, $\left< {\left(M^{\left(0, 1\right)}\right)}^{\mathtt{\text{2}}}\right> $, and $\left< M^{\left(1, 1\right)}\right> $.

Equations were iteratively added for $\left< N M^{\left(1, 0\right)}\right> $ and $\left< N M^{\left(0, 1\right)}\right> $.

d                                             
──(Expectation(N)) = -k_E⋅Expectation(N) + k_I
dt                                            

...

d                                                                                                                               
──(Expectation(Moment(0, 1))) = \lambda⋅k_I - k_E⋅Expectation(Moment(0, 1)) + k_b⋅Expectation(N) - k_d⋅Expectation(Moment(0, 1))
dt                                                                                                                              

                                                                                                                                                          2                   
d ⎛           ⎛      2      ⎞⎞          2                                                                                ⎛      2      ⎞   k_E⋅Expectation (Moment(0, 1))     
──⎝Expectation⎝Moment (0, 1)⎠⎠ = \lambda ⋅k_I + 2⋅\lambda⋅k_I⋅Expectation(Moment(0, 1)) + \lambda⋅k_I - 2⋅k_E⋅Expectation⎝Moment (0, 1)⎠ + ────────────────────────────── + k_
dt                                                                                                                                                 Expectation(N)             

                                                                                                                       
                                                                                                        ⎛      2      ⎞
b⋅Expectation(N) + 2⋅k_b⋅Expectation(N⋅Moment(0, 1)) + k_d⋅

                                                                                                                                                       ⎛       2              
d                               -k_E⋅Expectation(Moment(0, 1))⋅Expectation(Moment(1, 0)) - 2⋅k_d⋅Expectation(Moment(0, 1))⋅Expectation(Moment(1, 0)) + ⎝\lambda ⋅k_I + k_b⋅Exp
──(Expectation(Moment(1, 1))) = ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
dt                                                                                                                                 Expectation(N)                             

                                                      ⎞               
ectation(Moment(0, 1)) + k_b⋅Expectation(Moment(1, 0))⎠⋅Expectation(N)
──────────────────────────────────────────────────────────────────────
                                                                      

d                                                                                                                                                                             
──(Expectation(N⋅Moment(1, 0))) = \lambda⋅k_I⋅Expectation(N) + \lambda⋅k_I - 2⋅k_E⋅Expectation(N⋅Moment(1, 0)) + k_E⋅Expectation(Moment(1, 0)) + k_I⋅Expectation(Moment(1, 0))
dt                                                                                                                                                                            

                  ⎛ 2⎞                                  
 + k_b⋅Expectation⎝N ⎠ - k_d⋅Expectation(N⋅Moment(1, 0))
                                                        

d                                                                                                                                                                             
──(Expectation(N⋅Moment(0, 1))) = \lambda⋅k_I⋅Expectation(N) + \lambda⋅k_I - 2⋅k_E⋅Expectation(N⋅Moment(0, 1)) + k_E⋅Expectation(Moment(0, 1)) + k_I⋅Expectation(Moment(0, 1))
dt                                                                                                                                                                            

                  ⎛ 2⎞                                  
 + k_b⋅Expectation⎝N ⎠ - k_d⋅Expectation(N⋅Moment(0, 1))
                                                        

The LaTeX source of the ODE system can be found and copy-pasted by modifying the Math Render option of the jupyter notebook. This is done by right-clicking on the system and choosing Math Settings > Math Render > Plain Source .

From the closed equations, we can also generate code to simulate the system. Currently, Python or Julia code can be generated.

For a direct code output, the user can rely on the functions:

In [6]:
julia_code = generate_julia_code(equations, function_name="IEBDBD")
print(julia_code)

# evaluate ODEs
function IEBDBD_ODEs(dM, M, parameters, t)
  c2 = parameters[:\lambda] # \lambda
  c1 = parameters[:kE] # kE
  c4 = parameters[:kI] # kI
  c0 = parameters[:kb] # kb
  c3 = parameters[:kd] # kd
  # Number of Compartments (N)
  dM[1] = c4-c1*M[1]
  # N^2
  dM[2] = c4+c1*M[1]-2*c1*M[2]+2*c4*M[1]
  # Total Mass of species 1
  dM[3] = c2*c4+c0*M[1]-c1*M[3]-c3*M[3]
  # M1^2
  dM[4] = c2*c4+c4*((c2)^(2))+c0*M[1]+c3*M[3]-2*c1*M[4]-2*c3*M[4]+2*c0*M[8]+c1*((M[3])^(2))/M[1]+2*c2*c4*M[3]
  # Total Mass of species 2
  dM[5] = c2*c4+c0*M[1]-c1*M[5]-c3*M[5]
  # M2^2
  dM[6] = c2*c4+c4*((c2)^(2))+c0*M[1]+c3*M[5]-2*c1*M[6]-2*c3*M[6]+2*c0*M[9]+c1*((M[5])^(2))/M[1]+2*c2*c4*M[5]
  # Sum of squared content of species 1
  dM[7] = ((c4*((c2)^(2))+c0*M[5]+c0*M[3])*M[1]-c1*M[5]*M[3]-2*c3*M[5]*M[3])/M[1]
  # N*M1
  dM[8] = c2*c4+c1*M[3]+c4*M[3]+c0*M[2]-c3*M[8]-2*c1*M[8]+c2*c4*M[1]
  # N*M2
  dM[9] = c2*c4+c1*M[5]+c4*M[5]+c0*M[2]-c3*M[9]-2*c1*M[9]+c2*c4*M[1]
  return
end

# initialize expected mo